In [ ]:
from lec_utils import *

<div class="alert alert-info" markdown="1">

#### Lecture 17

# Multiple Linear Regression and Feature Engineering

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    TeX: {
      extensions: ["color.js"],
      packages: {"[+]": ["color"]},
    }
  });
  </script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-AMS_HTML"></script>

### Announcements 📣

- Homework 8 is due on **Friday** (not Thursday!).
- Homework 7 solutions are available at [**#259 on Ed**](https://edstem.org/us/courses/61012/discussion/5597496).
- Check out the new [**FAQs page on the course website**](https://practicaldsc.org/faqs).<br><small>It has answers to frequently-asked theoretical questions.</small>
- The IA application is out for next semester and is due on **Monday**! See [**#238 on Ed**](https://edstem.org/us/courses/61012/discussion/5563220) for more details.

### Agenda

- Recap: Regression and linear algebra.
- Multiple linear regression.
- Regression in `sklearn`.
- Feature engineering.
- Example: Horsepower 🚗.

Today's lecture is back to being in a notebook, but is still quite math heavy.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Recap: Regression and linear algebra

---

### Terminology recap

- Define the **design matrix** $\color{#007aff} X \in \mathbb{R}^{n \times 2}$, **observation vector** $\color{orange} \vec{y} \in \mathbb{R}^n$, and parameter vector $\vec{w} \in \mathbb{R}^2$ as:

$${\color{#007aff} {X} = \begin{bmatrix} {\color{#007aff} 1} & {\color{#007aff} x_1} \\ {\color{#007aff} 1} & {\color{#007aff} x_2} \\ \vdots & \vdots \\ {\color{#007aff} 1} & {\color{#007aff} x_n} \end{bmatrix}} \qquad {\color{orange} \vec{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}} \qquad \vec{w} = \begin{bmatrix} w_0 \\ w_1 \end{bmatrix}$$


- Our goal is to find the $\vec w^*$ that minimizes $R_\text{sq}(\vec{w}) = \frac{1}{n}  \lVert {\color{orange} \vec{y}} - {\color{#007aff} X}\vec{w} \rVert^2$.

- Last lecture, we showed that the best $\vec w^*$ is one that satisfies the **normal equations**:

$${\color{#007aff} X^TX} \vec{w}^* = {\color{#007aff} X^T} {\color{orange} \vec y}$$

- **If** ${\color{#007aff} X^TX}$ is invertible, there's a unique solution for $\vec w^*$:

$$\vec w^* = ({\color{#007aff}{X^TX}})^{-1} {\color{#007aff}{X^T}}\color{orange}\vec{y}$$

- We chose $\vec{w}^*$ so that ${\color{purple} \vec h^*} = {\color{#007aff} X} \vec{w}^*$ is **the projection of $\color{orange} \vec{y}$ onto the span of the columns of the design matrix, $\color{#007aff}X$**.

### The optimal parameter vector, $\vec{w}^*$

- To find the optimal model parameters for simple linear regression, $w_0^*$ and $w_1^*$, we previously minimized $R_\text{sq}(w_0, w_1) = \frac{1}{n} \sum_{i = 1}^n ({\color{orange} y_i} - (w_0 + w_1 {\color{#007aff} x_i}))^2$.

- We found, using calculus, that:
    - $\boxed{w_1^* = \frac{\sum_{i = 1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i = 1}^n (x_i - \bar{x})^2} = r \frac{\sigma_y}{\sigma_x}}$.
    - $\boxed{w_0^* = \bar{y} - w_1^* \bar{x}}$.

- Another way of finding optimal model parameters for simple linear regression is to find the $\vec{w}^*$ that minimizes $R_\text{sq}(\vec{w}) = \frac{1}{n}  \lVert {\color{orange} \vec{y}} - {\color{#007aff} X}\vec{w} \rVert^2$.

- The minimizer, if ${\color{#007aff}{X^TX}}$ is invertible, is the vector $\boxed{\vec{w}^* = ({\color{#007aff}{X^TX}})^{-1} {\color{#007aff}{X^T}}\color{orange}\vec{y}}$.<br><small>If not, solve the **normal equations** from the previous slide.</small>

- These $\boxed{\text{formulas}}$ are equivalent!

## Multiple linear regression

---

<center>

<img src="imgs/multiple-head.png" width=40%>

So far, we've fit **simple** linear regression models, which use only **one** feature (`'departure_hour'`) for making predictions.

</center>

### Incorporating multiple features

- In the context of the commute times dataset, the simple linear regression model we fit was of the form:

$$\begin{align*}\text{pred. commute} &= H(\text{departure hour}) \\ &= w_0 + w_1 \cdot \text{departure hour} \end{align*}$$

- Now, we'll try and fit a multiple linear regression model of the form:

$$\begin{align*}\text{pred. commute} &= H(\text{departure hour, day of month}) \\ &= w_0 + w_1 \cdot \text{departure hour} + w_2 \cdot \text{day of month} \end{align*}$$

- Linear regression with **multiple** features is called **multiple linear regression**.

- How do we find $w_0^*$, $w_1^*$, and $w_2^*$?


### Geometric interpretation

- The hypothesis function:

    $$H(\text{departure hour}) = w_0 + w_1 \cdot \text{departure hour}$$

    looks like a **line** in 2D.

- **Questions**: 
    - How many dimensions do we need to graph the hypothesis function:

    $$H(\text{departure hour, day of month}) = w_0 + w_1 \cdot \text{departure hour} + w_2 \cdot \text{day of month}$$

    - What is the shape of the hypothesis function?

<center>

<img src="imgs/plane.png" width=40%>

Our new hypothesis function is a **plane** in 3D!<br>Our goal is to find the **plane** of best fit that pierces through the cloud of points.

</center>

### The hypothesis vector

- When our hypothesis function is of the form:

    $$H(\text{departure hour}) = w_0 + w_1 \cdot \text{departure hour} + w_2 \cdot \text{day of month}$$

    the hypothesis vector $\color{purple} \vec{h} \in \mathbb{R}^n$ can be written as:

$${\color{purple} \vec{h}} = \begin{bmatrix}
    H(\text{departure hour}_1, \text{day}_1) \\
    H(\text{departure hour}_2, \text{day}_2) \\
        ... \\
     H(\text{departure hour}_n, \text{day}_n) \\
    \end{bmatrix}
    = \begin{bmatrix}
        1      & \text{departure hour}_1 & \text{day}_1   \\
        1      & \text{departure hour}_2 & \text{day}_2    \\
        ... & ... & ... \\
        1      & \text{departure hour}_n & \text{day}_n
    \end{bmatrix} \begin{bmatrix}
        w_0 \\
        w_1 \\
        w_2
    \end{bmatrix}  $$

### Finding the optimal parameters

- To find the optimal parameter vector, $\vec{w}^*$, we can use the **design matrix** $\color{#007aff} X \in \mathbb{R}^{n \times 3}$ and **observation vector** $\color{orange} \vec y \in \mathbb{R}^n$:

$${\color{#007aff} X = \begin{bmatrix}
        1      & \text{departure hour}_1 & \text{day}_1   \\
        1      & \text{departure hour}_2 & \text{day}_2    \\
        ... & ... & ... \\
        1      & \text{departure hour}_n & \text{day}_n
    \end{bmatrix}} \qquad {\color{orange} \vec{y} = \begin{bmatrix} \text{commute time}_1 \\ \text{commute time}_2 \\ \vdots \\ \text{commute time}_n \end{bmatrix}}$$

- Then, all we need to do is solve the normal equations once again:

    $${\color{#007aff} X^TX} \vec{w}^* = {\color{#007aff} X^T} {\color{orange} \vec y}$$

    If ${\color{#007aff} X^TX}$ is invertible, we know the solution is:

$$\vec{w}^* = ({\color{#007aff}{X^TX}})^{-1} {\color{#007aff}{X^T}}\color{orange}\vec{y}$$

- Let's generalize this notion beyond just two features.

### Notation for multiple linear regression

- We will need to keep track of multiple features for every individual in our dataset.<br><small>In practice, we could have hundreds, millions, or billions of features!</small>

- As before, subscripts distinguish between individuals in our dataset. We have $n$ individuals, also called **training examples**.

- Superscripts distinguish between **features**. We have $d$ features. 

    $$\text{departure hour:} \:\: \color{#007aff}x^{(1)}$$

    $$\text{day of month:} \:\: \color{#007aff}x^{(2)}$$

    Think of $\color{#007aff} x^{(1)}$, $\color{#007aff}x^{(2)}$, ... as new variable names, like new letters.

### Feature vectors

- Suppose we have the following dataset.

<center>

<img src="imgs/example-values.png" width=35%>

</center>

- We can represent each day with a **feature vector**, $\color{#007aff} \vec{x} = \begin{bmatrix} x^{(1)} \\ x^{(2)} \end{bmatrix}$:

$$\color{#007aff} \vec x_1 = \begin{bmatrix} 8.45 \\ 22 \end{bmatrix} \qquad \vec x_2 = \begin{bmatrix} 8.90 \\ 28 \end{bmatrix} \qquad \vec x_2 = \begin{bmatrix} 8.72 \\ 18 \end{bmatrix}$$

### Augmented feature vectors

- The **augmented feature vector** $\text{Aug}({\color{#007aff}\vec x})$ is the vector obtained by adding a 1 to the front of feature vector $\color{#007aff} \vec x$:

$$
{\color{#007aff} \vec x = \begin{bmatrix}
    {\color{#007aff}x^{(1)}} \\ {\color{#007aff}x^{(2)}} \\ \vdots \\ {\color{#007aff}x^{(d)}}
\end{bmatrix}
\qquad
\text{Aug}({\color{#007aff} \vec x}) = \begin{bmatrix}
    1 \\ {\color{#007aff}x^{(1)}} \\ {\color{#007aff}x^{(2)}} \\ \vdots \\ {\color{#007aff} x^{(d)}}
\end{bmatrix}}
\qquad
\vec w = \begin{bmatrix}
    w_0 \\ w_1 \\ w_2\\ \vdots \\ w_d 
\end{bmatrix}
$$

- For example, if $\color{#007aff} \vec x_1 = \begin{bmatrix} 8.45 \\ 22 \end{bmatrix}$, then $\text{Aug}({\color{#007aff} \vec x_1}) = {\color{#007aff} \begin{bmatrix} 1 \\ 8.45 \\ 22 \end{bmatrix}}$.

- Then, our hypothesis function for a single data point is:
$$\begin{align*}
                H({\color{#007aff} \vec x}) &= w_0 + w_1 {\color{#007aff} x^{(1)}} + w_2 {\color{#007aff} x^{(2)}} + \ldots + w_d {\color{#007aff} x^{(d)}}\\
                               &=    \vec w \cdot \text{Aug}({\color{#007aff} \vec x})
\end{align*}$$

### The general problem

-  We have $n$ data points, $\left({\color{#007aff} \vec x_1}, {\color{orange}y_1}\right), \left({\color{#007aff} \vec x_2}, {\color{orange}y_2}\right),  \ldots, \left({\color{#007aff} \vec x_n}, {\color{orange}y_n}\right)$,
where each $\color{#007aff} \vec x_i$ is a feature vector of $d$ features:
$${\color{#007aff}\vec{x_i}} = \begin{bmatrix} 
{\color{#007aff}x^{(1)}_i} \\ {\color{#007aff}x^{(2)}_i} \\ \vdots \\ {\color{#007aff}x^{(d)}_i}
\end{bmatrix}$$	   

-  We want to find a good linear hypothesis function:
$$\begin{align*}
                H({\color{#007aff} \vec x}) &= w_0 + w_1 {\color{#007aff} x^{(1)}} + w_2 {\color{#007aff} x^{(2)}} + \ldots + w_d {\color{#007aff} x^{(d)}}\\
                               &=    \vec w \cdot \text{Aug}({\color{#007aff} \vec x})
\end{align*}$$

### The general solution

- Define the **design matrix** $ X \in \mathbb{R}^{n \times (d + 1)}$ and **observation vector** $\color{orange} \vec y \in \mathbb{R}^n$:

$${\color{#007aff} X=  \begin{bmatrix}  
{1} & { x^{(1)}_1} & { x^{(2)}_1} & \dots & { x^{(d)}_1} \\\\
{ 1} & { x^{(1)}_2} & { x^{(2)}_2} & \dots & { x^{(d)}_2} \\\\
\vdots & \vdots & \vdots  &  & \vdots \\\\
{ 1} & { x^{(1)}_n} & { x^{(2)}_n} & \dots & { x^{(d)}_n}
\end{bmatrix} = \begin{bmatrix} 
       \text{Aug}({\vec{x_1}})^T \\\\
       \text{Aug}({\vec{x_2}})^T \\\\
       \vdots \\\\
       \text{Aug}({\vec{x_n}})^T
   \end{bmatrix}} \qquad {\color{orange} \vec y = \begin{bmatrix} {\color{orange} y_1} \\ {\color{orange} y_2} \\ \vdots \\ {\color{orange} y_n} \end{bmatrix}}$$

- Then, solve the **normal equations** to find the optimal parameter vector, $\vec{w}^*$:

    $${\color{#007aff} X^TX} \vec{w}^* = {\color{#007aff} X^T} {\color{orange} \vec y}$$


### Note on parameters

-  With $d$ features, $\vec w$ has $d+1$ entries.

- Again, $\vec w^*$ represents our **optimal** parameter vector, that minimizes mean squared error.

-  $w_0^*$ is the **bias**, also known as the **intercept**.

-  $w_1^*, w_2^*, ... , w_d^*$ each give the **weight**, or **coefficient**, or **slope**, of a feature.

<br>

$$H^*({\color{#007aff} \vec x}) = w_0^* + w_1^* {\color{#007aff} x^{(1)}} + w_2^* {\color{#007aff} x^{(2)}} + \ldots + w_d^* {\color{#007aff} x^{(d)}}$$

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Regression in `sklearn`

### Loading the data

- Run the cell below to load in our commute times dataset.

In [ ]:
df = pd.read_csv('data/commute-times.csv')
df['day_of_month'] = pd.to_datetime(df['date']).dt.day
df.head()

- For now, the only relevant columns for us are `'departure_hour'`, `'day_of_month'`, and `'minutes'`.

In [ ]:
df[['departure_hour', 'day_of_month', 'minutes']]

### `sklearn`

- `sklearn` (scikit-learn) implements many common steps in the feature and model creation pipeline.<br><small>It is **widely** used throughout [industry](https://scikit-learn.org/stable/testimonials/testimonials.html#:~:text=It%20is%20very%20widely%20used,very%20approachable%20and%20very%20powerful.) and academia.</small>

<center><img src='imgs/sklearn.png' width=20%></center>

- It interfaces with `numpy` arrays, and to an extent, `pandas` DataFrames.

- Huge benefit: the [documentation online](https://scikit-learn.org/stable/modules/classes.html) is excellent.

### The `LinearRegression` class

- `sklearn` comes with several subpackages, including `linear_model` and `tree`, each of which contains several classes of models.

- We'll start with the `LinearRegression` class from `linear_model`.

In [ ]:
from sklearn.linear_model import LinearRegression

- **Important**: From [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression), we have:
> LinearRegression fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

    In other words, **`LinearRegression` minimizes mean squared error by default**! (Per the documentation, it also includes an intercept term by default.)

In [ ]:
LinearRegression?

### Fitting a multiple linear regression model

- Let's aim to use `sklearn` to find the optimal parameters for the following model:

    $$\text{predicted commute time} = w_0 + w_1 \cdot \text{departure hour} + w_2 \cdot \text{day of month}$$

- First, we must instantiate a `LinearRegression` object and fit it. By calling `fit`, we are saying "minimize mean squared error on this dataset and find $w^*$."

In [ ]:
model_multiple = LinearRegression()
# Note that there are two arguments to fit – X and y!
# (It is not necessary to write X= and y=)
...

- After fitting, we can access $w^*$ – that is, the best intercept and coefficients.

In [ ]:
...

- These coefficients tell us that the "best way" (according to squared loss) to make commute time predictions using a linear model is using:

$$\text{predicted commute time} = 141.86 - 8.22 \cdot \text{departure hour} + 0.06 \cdot \text{day of month}$$

- This is the **plane of best fit** given historical data; it is not a causal statement.

- Let's visualize this model, as we did in Lecture 16.

In [ ]:
XX, YY = np.mgrid[5:14:1, 0:31:1]
Z = model_multiple.intercept_ + model_multiple.coef_[0] * XX + model_multiple.coef_[1] * YY
plane = go.Surface(x=XX, y=YY, z=Z, colorscale='Reds')
fig = go.Figure(data=[plane])
fig.add_trace(go.Scatter3d(x=df['departure_hour'], 
                           y=df['day_of_month'], 
                           z=df['minutes'], mode='markers', marker = {'color': '#656DF1'}))
fig.update_layout(scene=dict(xaxis_title='Departure Hour',
                             yaxis_title='Day of Month',
                             zaxis_title='Minutes'),
                  title='Commute Time vs. Departure Hour and Day of Month',
                  width=1000, height=500)

### Making predictions

- Fit `LinearRegression` objects also have a `predict` method, which can be used to predict commute times given a `'departure_hour'` and `'day_of_month'`.

In [ ]:
# What if I leave at 9:15AM on the 26th of the month?
...

In [ ]:
# Since we trained on a DataFrame, the input to model_multiple.predict should also
# be a DataFrame. To avoid having to do this, we can use .to_numpy()
# when specifying X= and y=.
model_multiple.predict(pd.DataFrame({'departure_hour': [9.25], 'day_of_month': [26]}))

### Comparing models

- Since we're going to start to fit lots of different models to the commute times dataset, it'll be useful to compare their mean squared errors.

- `sklearn` has a built-in `mean_squared_error` function.<br><small>Remember, the units of MSE are the units of $y$, squared. So the value below is 96.78 $\text{minutes}^2$.</small>

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
...

- Let's construct a dictionary to keep track of the MSEs we've seen so far.

In [ ]:
mse_dict = {}
mse_dict['departure_hour + day_of_month'] = mean_squared_error(df['minutes'], model_multiple.predict(df[['departure_hour', 'day_of_month']]))

- To compare, let's also fit and measure a simple linear model and constant model.

In [ ]:
# Simple linear model.
model_simple = LinearRegression()
model_simple.fit(X=df[['departure_hour']], y=df['minutes'])
mse_dict['departure_hour'] = mean_squared_error(df['minutes'], model_simple.predict(df[['departure_hour']]))

In [ ]:
# Constant model.
model_constant = df['minutes'].mean()
mse_dict['constant'] = mean_squared_error(df['minutes'], np.ones(df.shape[0]) * model_constant)

- As we can see, adding `'day_of_month'` as a feature **barely** reduced our model's MSE.<br><small>Next week, when we learn about generalization, we'll see why sometimes adding more features is a **bad** thing!</small>

In [ ]:
mse_dict

### The `.score` method of a `LinearRegression` object

- Model objects in `sklearn` that have already been fit have a `score` method.

In [ ]:
...

- That doesn't look like the RMSE... what is it? 🤔

### Aside: $R^2$

- $R^2$, pronounced "**coefficient of determination**" or "**multiple R-squared**", is a measure of the **quality of a linear fit**.

- There are a few equivalent ways of computing it, assuming your model **is linear and has an intercept term**:

$$R^2 = \frac{\text{var}(\text{predicted $y$ values})}{\text{var}(\text{actual $y$ values})}$$

$$R^2 = \left[ \text{correlation}(\text{predicted $y$ values}, \text{actual $y$ values}) \right]^2$$

- Interpretation: $R^2$ is the **proportion of variance in $y$ that the linear model explains**.

- In the simple linear regression case, it is the square of the correlation coefficient, $r$.

- **Key idea:** $R^2$ ranges from 0 to 1. **The closer it is to 1, the better the linear fit is.**<br><small>Just like $r$, $R^2$ has no units of measurement. This is unlike MSE, which has the units of $y$, squared.</small>

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Calculating $R^2$

- Let's calculate the $R^2$ for `model_multiple`'s predictions in three different ways.

In [ ]:
pred = df.assign(predicted=model_multiple.predict(df[['departure_hour', 'day_of_month']]))
pred

- **Method 1: $R^2 = \frac{\text{var}(\text{predicted $y$ values})}{\text{var}(\text{actual $y$ values})}$**.

In [ ]:
np.var(pred['predicted']) / np.var(pred['minutes'])

- **Method 2:** $R^2 = \left[ \text{correlation}(\text{predicted $y$ values}, \text{actual $y$ values}) \right]^2$.<br><small>Note: By correlation here, we are referring to $r$, the same correlation coefficient we saw last week.</small>

In [ ]:
np.corrcoef(pred['predicted'], pred['minutes'])[0, 1] ** 2

- **Method 3:** `LinearRegression.score`.

In [ ]:
model_multiple.score(df[['departure_hour', 'day_of_month']], df['minutes'])

- All three methods provide the same result!

### Relationship between $R^2$ and MSE

- For linear models with an intercept term,

$$R^2 = 1 - \frac{\text{MSE}}{\text{var}(\text{actual $y$ values})}$$

In [ ]:
1 - mean_squared_error(pred['minutes'], pred['predicted']) / np.var(pred['minutes'])

### `LinearRegression` class summary

|Property|Example|Description|
|---|---|---|
|Initialize model parameters| `lr = LinearRegression()` | Create (empty) linear regression model|
|Fit the model to the data | `lr.fit(X, y)` | Determines regression coefficients|
|Use model for prediction |`lr.predict(X_new)`| Uses regression line to make predictions|
|Evaluate the model| `lr.score(X, y)` | Calculates the $R^2$ of the LR model|
|Access model attributes| `lr.coef_`, `lr.intercept_` | Accesses the regression coefficients and intercept|


### What's next?

- So far, in our journey to predict `'minutes'`, we've only used two numerical features in our dataset, `'departure_hour'` and `'day_of_month'`.

In [ ]:
df[['departure_hour', 'day_of_month', 'minutes']]

- There's a lot of information in `df` that we didn't use –`'day'`, for example. We can't use these `'day'` in it's current form as it's non-numeric.

- **How do we use categorical features in a regression model?**

## Feature engineering ⚙️

### The goal of feature engineering

- **Feature engineering** is the act of finding **transformations** that transform data into effective **quantitative variables**.

- A feature function $\phi$ (phi, pronounced "fea") is a mapping from raw data to $d$-dimensional space, i.e. $\phi: \text{raw data} \rightarrow \mathbb{R}^d$.<br><small>If two observations $x_i$ and $x_j$ are "similar" in the raw data space, then $\phi(x_i)$ and $\phi(x_j)$ should also be "similar."</small>

- A "good" choice of features depends on many factors:
    - The kind of data, i.e. quantitative, ordinal, or nominal.
    - The relationship(s) being modeled.
    - The model type, e.g. linear models, decision tree models, neural networks.

### One hot encoding

- One hot encoding is a transformation that turns a categorical feature into several binary features.

- Suppose a column has $N$ unique values, $A_1$, $A_2$, ..., $A_N$. For each unique value $A_i$, we define the following **feature function**:

$$\phi_i(x) = \left\{\begin{array}{ll}1 & {\rm if\ } x == A_i \\ 0 &  {\rm if\ } x\neq A_i \\ \end{array}\right. $$

- Note that 1 means "yes" and 0 means "no".

- One hot encoding is also called "dummy encoding", and $\phi(x)$ may also be referred to as an "indicator variable".

### Example: One hot encoding `'day'`

- For each unique value of `'day'` in our dataset, we must create a column for just that `'day'`.

In [ ]:
df.head()

In [ ]:
df['day'].value_counts()

In [ ]:
...

In [ ]:
for val in df['day'].unique():
    df[f'day == {val}'] = (df['day'] == val).astype(int)

In [ ]:
...

### Using `'day'` as a feature, along with `'departure_hour'` and `'day_of_month'`

- Now that we've converted `'day'` to a numerical variable, we can use it as input in a regression model. Here's the model we'll try to fit:

$$\begin{align*}\text{predicted commute time} = w_0 &+ w_1 \cdot \text{departure hour} \\ &+ w_2 \cdot \text{day of month} \\ &+ w_3 \cdot \text{day == Mon} \\ 
&+ w_4 \cdot \text{day == Tue} \\ 
&+ w_5 \cdot \text{day == Wed} \\ 
&+ w_6 \cdot \text{day == Thu} \end{align*}$$

- **Subtlety**: Since there are only 5 values of `'day'`, we don't need to include `'day == Fri'` as a feature. We know it's Friday if `'day == Mon'`, ..., `'day == Thu'` are all 0.<br><small>More on why next class!</small>

In [ ]:
X_for_ohe = df[['departure_hour', 
                         'day_of_month',
                         'day == Mon',
                         'day == Tue',
                         'day == Wed',
                         'day == Thu']]
X_for_ohe

In [ ]:
model_with_ohe = LinearRegression()
model_with_ohe.fit(X=X_for_ohe, y=df['minutes'])

- The following cell gives us our $w^*$s:

In [ ]:
model_with_ohe.intercept_, model_with_ohe.coef_

- Thus, our trained linear model to predict commute time given `'departure_hour'`, `'day_of_month'`, and `'day'` (Mon, Tue, Wed, or Thu) is:

$$\begin{align*}\text{predicted commute time} = 134 &- 8.42 \cdot \text{departure hour} \\ &- 0.03 \cdot \text{day of month} \\ 
&+ 5.09 \cdot \text{day == Mon} \\ 
&+ 16.38 \cdot \text{day == Tue} \\ 
&+ 5.12 \cdot \text{day == Wed} \\ 
&+ 11.5 \cdot \text{day == Thu} \end{align*}$$

### Visualizing our latest model

- Our trained linear model to predict commute time given `'departure_hour'`, `'day_of_month'`, and `'day'` (Mon, Tue, Wed, or Thu) is:

$$\begin{align*}\text{predicted commute time} = 134 &- 8.42 \cdot \text{departure hour} \\ &- 0.03 \cdot \text{day of month} \\ 
&+ 5.09 \cdot \text{day == Mon} \\ 
&+ 16.38 \cdot \text{day == Tue} \\ 
&+ 5.12 \cdot \text{day == Wed} \\ 
&+ 11.5 \cdot \text{day == Thu} \end{align*}$$

- Since we have 6 features here, we'd need 7 dimensions to graph our model.

- But, as we see in Homework 8, Question 6, our model is really a collection of **five parallel planes** in 3D, all with slightly different $z$-intercepts!

- If we want to visualize in 2D, we need to pick a single feature to place on the $x$-axis.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['departure_hour'], y=df['minutes'], 
                         mode='markers', name='Original Data'))
fig.add_trace(go.Scatter(x=df['departure_hour'], y=model_with_ohe.predict(X_for_ohe), 
                         mode='markers', name='Predicted Commute Times using Departure Hour, <br>Day of Month, and Day of Week'))
fig.update_layout(showlegend=True, title='Commute Time vs. Departure Hour',
                  xaxis_title='Departure Hour', yaxis_title='Minutes', width=1000)

- Despite being a linear model, why **doesn't** this model **look** like a straight line?

### Comparing our latest model to earlier models

- Let's see how the inclusion of the day of the week impacts the quality of our predictions.

In [ ]:
mse_dict['departure_hour + day_of_month + ohe day'] = mean_squared_error(
    df['minutes'],
    model_with_ohe.predict(X_for_ohe)
)

In [ ]:
mse_dict

- Adding the day of the week decreased our MSE **significantly**!

### Reflection

In [ ]:
df.head()

- We've one hot encoded `'day'`, but it required a `for`-loop.

- Is there a way we could have encoded it without a `for`-loop?

- Yes, using `sklearn.preprocessing`'s `OneHotEncoder`. More on this soon!

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Example: Horsepower 🚗

---

### Loading the (new) data

- The following dataset, built into the `seaborn` plotting library, contains various information about (older) cars.

In [ ]:
mpg = sns.load_dataset('mpg').dropna()
mpg.head()

- We really do mean old:

In [ ]:
mpg['model_year'].value_counts()

- Let's investigate the relationship between `'horsepower'` and `'mpg'`.

### The relationship between `'horsepower'` and `'mpg'`

In [ ]:
px.scatter(mpg, x='horsepower', y='mpg')

- It appears that there is a negative association between `'horsepower'` and `'mpg'`, though it's not quite linear.

- Let's try and fit a simple linear model that uses `'horsepower'` to predict `'mpg'` and see what happens.

### Predicting `'mpg'` using `'horsepower'`

In [ ]:
car_model = LinearRegression()
car_model.fit(mpg[['horsepower']], mpg['mpg'])

- What do our predictions look like?

In [ ]:
hp_points = pd.DataFrame({'horsepower': [25, 225]})
fig = px.scatter(mpg, x='horsepower', y='mpg')
fig.add_trace(go.Scatter(
    x=hp_points['horsepower'],
    y=car_model.predict(hp_points),
    mode='lines',
    name='Predicted MPG using Horsepower'
))

- Our regression line doesn't capture the curvature in the relationship between `'horsepower'` and `'mpg'`.

In [ ]:
car_model.score(mpg[['horsepower']], mpg['mpg'])

### Linear in the parameters

-  **Using linear regression**, we can fit rules like:
    $$
    w_0 + w_1x+w_2x^2
    \qquad \qquad 
    w_1e^{-x^{{(1)}^2}} + w_2 \cos(x^{(2)}+\pi) +w_3 \frac{\log 2x^{(3)}}{x^{(2)}}
    $$
    - This includes arbitrary polynomials.
    - These are all linear combinations of (just) features.

- For any of the above examples, we **could** express our model as a product of a design matrix and parameter vector, and that's all that `LinearRegression` in `sklearn` needs.<br><small>What we put in the `X` argument to `model.fit` is up to us!</small>

-  Using linear regression, we **can't** fit rules like:
    $$
    w_0 + e^{w_1 x}
    \qquad \qquad 
    w_0 + \sin (w_1 x^{(1)} + w_2 x^{(2)})
    $$
    - These are **not** linear combinations of just features.

-  We can have any number of parameters, as long as our hypothesis function is **linear in the parameters**, or linear when we think of it as a function of the parameters.

$$w_0 + w_1 f_1(\vec{x}) + w_2 f_2(\vec{x}) + ... + w_d f_d(\vec{x})$$

### Linearization

- The [Tukey Mosteller Bulge Diagram](https://sites.stat.washington.edu/pds/stat423/Documents/LectureNotes/notes.423.ch4.pdf) helps us pick which transformations to apply to data in order to **linearize** it.

<center><img src="imgs/bulge.png" width=25%></center>

- The bottom-left quadrant appears to match the shape of the scatter plot between `'horsepower'` and `'mpg'` the best – let's try taking the `log` of `'horsepower'` ($X$).

In [ ]:
mpg['log hp'] = np.log(mpg['horsepower'])

- What does our data look like now?

In [ ]:
px.scatter(mpg, x='log hp', y='mpg')

### Predicting `'mpg'` using `log('horsepower')`

- Let's fit another linear model.

In [ ]:
car_model_log = LinearRegression()
car_model_log.fit(mpg[['log hp']], mpg['mpg'])

- Note that implicitly, we defined the following design matrix:

$$X = \begin{bmatrix} 1 & \log(x_1) \\ 1 & \log(x_2) \\ \vdots & \vdots \\ 1 & \log(x_n) \end{bmatrix}$$

- What do our predictions look like now?

In [ ]:
fig = px.scatter(mpg, x='log hp', y='mpg')
log_hp_points = pd.DataFrame({'log hp': [3.7, 5.5]})
fig = px.scatter(mpg, x='log hp', y='mpg')
fig.add_trace(go.Scatter(
    x=log_hp_points['log hp'],
    y=car_model_log.predict(log_hp_points),
    mode='lines',
    name='Predicted MPG using log(Horsepower)'
))

- The fit looks a bit better! How about the $R^2$?

In [ ]:
car_model_log.score(mpg[['log hp']], mpg['mpg'])

- Also a bit better!

- What do our predictions look like on the original, non-transformed scatter plot? Let's see:

In [ ]:
fig = px.scatter(mpg, x='horsepower', y='mpg')
fig.add_trace(
    go.Scatter(
        x=mpg['horsepower'], 
        y=car_model_log.intercept_ + car_model_log.coef_[0] * np.log(mpg['horsepower']),  
        mode='markers', name='Predicted MPG using log(Horsepower)'
    )
)
fig

- Our predictions that used $\log(\text{Horsepower})$ as an input don't fall on a straight line. We shouldn't expect them to; the orange dots come from:

$$\text{predicted MPG} = 108.70 - 18.582 \cdot \log(\text{Horsepower})$$

In [ ]:
car_model_log.intercept_, car_model_log.coef_

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>

Which hypothesis function is **not** linear in the parameters?

- A. $H(\vec{x}) = w_1 (x^{(1)} x^{(2)}) + \frac{w_2}{x^{(1)}} \sin \left( x^{(2)} \right)$
- B. $H(\vec{x}) = 2^{w_1} x^{(1)}$
- C. $H(\vec{x}) = \vec{w} \cdot \text{Aug}(\vec{x})$
- D. $H(\vec{x}) = w_1 \cos (x^{(1)}) + w_2 2^{x^{(2)} \log x^{(3)}}$
- E. More than one of the above.

### What's next?

- Next class, we'll look at more examples of feature transformations.

- We'll also look at how to implement more of the feature engineering process in `sklearn`, ultimately creating **Pipeline** objects that both:
    - Define a model.
    - Define the necessary feature transformations to fit the model.